In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.linear_model import Ridge
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline

In [2]:
train_data = pd.read_csv('data/train.csv', index_col='customer')
test_data = pd.read_csv('data/test.csv', index_col='customer')

In [3]:
train_dummies = pd.get_dummies(train_data, columns=['category', 'is_pep', 'nationality'])
test_dummies = pd.get_dummies(test_data, columns=['category', 'is_pep', 'nationality'])

In [4]:
x_train = train_dummies.drop(['suspicious'], axis=1)
y_train = pd.DataFrame(train_dummies['suspicious'])
x_test = test_dummies.copy()
x_test['nationality_117'] = 0

In [5]:
pipeline = Pipeline([
    ('normalizer', Normalizer()), 
    ('ridge', Ridge(alpha=1.0))
])

In [6]:
pipeline.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('normalizer', Normalizer(copy=True, norm='l2')), ('ridge', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

In [7]:
y_pred = pipeline.predict(x_test)

In [8]:
np.max(y_pred)

0.039636925121546096